## Libraries

In [ ]:
#install the Pyterrier framework
!pip install python-terrier --quiet

# install the nltk modules
!pip install nltk --quiet

# Need to install additional terrier package for PRF. It will take around 1 min
!git clone https://github.com/terrierteam/terrier-prf/
!apt-get install maven   #used for Java projects to manage project dependencies and build processes
%cd /content/terrier-prf/
!mvn install
!pwd
%cd ..

!pip install flask_ngrok
!pip install --ignore-installed blinker
!pip install git+https://github.com/experimaestro/experimaestro-ir.git --quiet

#install the transformer library
!pip install transformers
!pip install flask

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.9/337.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py)

In [ ]:
import pandas as pd
import pyterrier as pt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
import os
pd.set_option('display.max_colwidth', 150)
from collections import defaultdict
import math
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import gensim
from gensim.models import Word2Vec
import time
from flask import Flask, request, render_template
from flask import Flask,render_template,request
from flask_ngrok import run_with_ngrok

## Preprocess

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
print(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
# Initialize Porter stemmer
stemmer = PorterStemmer()

In [ ]:
def Stem_text(text):
    tokens = word_tokenize(text)
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(stemmed_tokens)

def clean(text):
    text = re.sub(r"[\.\,\#_\|\:\?\?\/\=\@]", " ", text)  # remove special characters
    text = re.sub(r'\t', ' ', text)  # remove tabs
    text = re.sub(r'\n', ' ', text)  # remove line jump
    text = re.sub(r"\s+", " ", text)  # remove extra white space
    text = text.strip()
    return text

def remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word.lower() for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered_tokens)

def preprocess(sentence):
    sentence = clean(sentence)
    sentence = remove_stopwords(sentence)
    sentence = Stem_text(sentence)
    return sentence


## Load Dataset

In [ ]:
import zipfile
zip_file_name = 'cisi.zip'
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall('cisi_dataset')
!ls cisi_dataset

CISI.ALL  CISI.QRY  CISI.REL


In [ ]:
def load_cisi_dataset(data_dir):
    documents_path = os.path.join(data_dir, 'CISI.ALL')
    queries_path = os.path.join(data_dir, 'CISI.QRY')
    qrels_path = os.path.join(data_dir, 'CISI.REL')

    doc_df = read_documents(documents_path)
    queries_df = read_queries(queries_path)
    qrels_df = read_qrels(qrels_path)
    return doc_df, queries_df, qrels_df

# Read documents from CISI.ALL file
def read_documents(documents_path):
    with open(documents_path, 'r') as file:
        lines = file.readlines()
    documents = []
    current_document = None
    for line in lines:
        if line.startswith('.I'):
            if current_document is not None:
                current_document['Text'] = current_document['Text'].split('\t')[0].strip()  # Remove anything after the first tab
                documents.append(current_document)
            current_document = {'ID': line.strip().split()[1], 'Text': ''}
        elif line.startswith('.T'):
            continue
        elif line.startswith('.A') or line.startswith('.B') or line.startswith('.W') or line.startswith('.X'):
            continue
        else:
            current_document['Text'] += line.strip() + ' '

    # Append the last document
    if current_document is not None:
        current_document['Text'] = current_document['Text'].split('\t')[0].strip()  # Remove anything after the first tab
        documents.append(current_document)
    doc_df = pd.DataFrame(documents)
    return doc_df

# Read queries from CISI.QRY file
def read_queries(queries_path):
    with open(queries_path, 'r') as file:
        lines = file.readlines()
    query_texts = []
    query_ids = []
    current_query_id = None
    current_query_text = []
    for line in lines:
        if line.startswith('.I'):
            if current_query_id is not None:
                query_texts.append(' '.join(current_query_text))
                current_query_text = []
            current_query_id = line.strip().split()[1]
            query_ids.append(current_query_id)
        elif line.startswith('.W'):
            continue
        elif line.startswith('.X'):
            break
        else:
            current_query_text.append(line.strip())
    # Append the last query
    query_texts.append(' '.join(current_query_text))
    queries_df = pd.DataFrame({
        'qid': query_ids,
        'raw_query': query_texts})
    return queries_df

# Read qrels from CISI.REL file
def read_qrels(qrels_path):
    qrels_df = pd.read_csv(qrels_path, sep='\s+', names=['qid','Q0','docno','label'])
    return qrels_df

In [ ]:
data_dir = '/content/cisi_dataset'
doc_df, queries_df, qrels_df = load_cisi_dataset(data_dir)

In [ ]:
doc_df

,ID,Text
0,1,"18 Editions of the Dewey Decimal Classifications Comaromi, J.P. The present study is a history of the DEWEY Decimal Classification. The first edi..."
1,2,"Use Made of Technical Libraries Slater, M. This report is an analysis of 6300 acts of use in 104 technical libraries in the United Kingdom. Librar..."
2,3,"Two Kinds of Power An Essay on Bibliographic Control Wilson, P. The relationships between the organization and control of writings and the organiz..."
3,4,"Systems Analysis of a University Library; final report and research project Buckland, M.K. The establishment of nine new universities in the 1960'..."
4,5,"A Library Management Game: a report on a research project Brophy, P. Although the use of games in professional education has become widespread onl..."
...,...,...
1455,1456,"World Dynamics Forrester, J.W. Over the last several decades interest in economic development, population growth, and the world environment has ex..."
1456,1457,"World Trends in Library Education Bramley, G. One of the most significant aspects of the evolution of librarianship in the twentieth century has b..."
1457,1458,"Legal Restrictions on Exploitation of the Patent Monopoly: An Economic Analysis Baxter, W.A. The patent laws confer on a patentee power to exclude..."
1458,1459,"Language and Thought Poluskin, V.A. This book considers the basic aspects of this complex problem - the historical and social essence of language ..."


In [ ]:
queries_df

,qid,raw_query
0,1,What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from app...
1,2,"How can actually pertinent data, as opposed to references or entire articles themselves, be retrieved automatically in response to information req..."
2,3,What is information science? Give definitions where possible.
3,4,Image recognition and any other methods of automatically transforming printed text into computer-ready form.
4,5,What special training will ordinary researchers and businessmen need for proper information management and unobstructed use of information retriev...
...,...,...
107,108,".T A Program for Machine-Mediated Searching .A Toliver, D. A technique of online instruction and assistance to bibliographic data base searchers c..."
108,109,".T Author Cocitation: A Literature Measure of Intellectual Structure .A White, H.D. Griffith, B.C. It is shown that the mapping of a particular a..."
109,110,".T Progress in Documentation. Word Processing: An Introduction and Appraisal .A Whitehead, J. The ""Office of the Future,"" ""Office Technology,"" ""W..."
110,111,".T Document Clustering Using an Inverted File Approach .A Willett, P. An automated document clustering procedure is described which does not requi..."


In [ ]:
qrels_df

,qid,Q0,docno,label
0,1,28,0,0.0
1,1,35,0,0.0
2,1,38,0,0.0
3,1,42,0,0.0
4,1,43,0,0.0
...,...,...,...,...
3109,111,422,0,0.0
3110,111,448,0,0.0
3111,111,485,0,0.0
3112,111,503,0,0.0


## Editing Dataset

In [ ]:
#the docno will be our ID
doc_df["docno"]=doc_df["ID"].astype(str)
doc_df

,ID,Text,docno
0,1,"18 Editions of the Dewey Decimal Classifications Comaromi, J.P. The present study is a history of the DEWEY Decimal Classification. The first edi...",1
1,2,"Use Made of Technical Libraries Slater, M. This report is an analysis of 6300 acts of use in 104 technical libraries in the United Kingdom. Librar...",2
2,3,"Two Kinds of Power An Essay on Bibliographic Control Wilson, P. The relationships between the organization and control of writings and the organiz...",3
3,4,"Systems Analysis of a University Library; final report and research project Buckland, M.K. The establishment of nine new universities in the 1960'...",4
4,5,"A Library Management Game: a report on a research project Brophy, P. Although the use of games in professional education has become widespread onl...",5
...,...,...,...
1455,1456,"World Dynamics Forrester, J.W. Over the last several decades interest in economic development, population growth, and the world environment has ex...",1456
1456,1457,"World Trends in Library Education Bramley, G. One of the most significant aspects of the evolution of librarianship in the twentieth century has b...",1457
1457,1458,"Legal Restrictions on Exploitation of the Patent Monopoly: An Economic Analysis Baxter, W.A. The patent laws confer on a patentee power to exclude...",1458
1458,1459,"Language and Thought Poluskin, V.A. This book considers the basic aspects of this complex problem - the historical and social essence of language ...",1459


In [ ]:
queries_df["qid"]=queries_df["qid"].astype(str)
queries_df

,qid,raw_query
0,1,What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from app...
1,2,"How can actually pertinent data, as opposed to references or entire articles themselves, be retrieved automatically in response to information req..."
2,3,What is information science? Give definitions where possible.
3,4,Image recognition and any other methods of automatically transforming printed text into computer-ready form.
4,5,What special training will ordinary researchers and businessmen need for proper information management and unobstructed use of information retriev...
...,...,...
107,108,".T A Program for Machine-Mediated Searching .A Toliver, D. A technique of online instruction and assistance to bibliographic data base searchers c..."
108,109,".T Author Cocitation: A Literature Measure of Intellectual Structure .A White, H.D. Griffith, B.C. It is shown that the mapping of a particular a..."
109,110,".T Progress in Documentation. Word Processing: An Introduction and Appraisal .A Whitehead, J. The ""Office of the Future,"" ""Office Technology,"" ""W..."
110,111,".T Document Clustering Using an Inverted File Approach .A Willett, P. An automated document clustering procedure is described which does not requi..."


In [ ]:
doc_df['processed_text'] = doc_df['Text'].apply(preprocess)
doc_df

,ID,Text,docno,processed_text
0,1,"18 Editions of the Dewey Decimal Classifications Comaromi, J.P. The present study is a history of the DEWEY Decimal Classification. The first edi...",1,18 edit dewey decim classif comaromi j p present studi histori dewey decim classif first edit ddc publish 1876 eighteenth edit 1971 futur edit con...
1,2,"Use Made of Technical Libraries Slater, M. This report is an analysis of 6300 acts of use in 104 technical libraries in the United Kingdom. Librar...",2,use made technic librari slater report analysi 6300 act use 104 technic librari unit kingdom librari use one aspect wider pattern inform use infor...
2,3,"Two Kinds of Power An Essay on Bibliographic Control Wilson, P. The relationships between the organization and control of writings and the organiz...",3,two kind power essay bibliograph control wilson p relationship organ control write organ control knowledg inform inevit enter stori write contain ...
3,4,"Systems Analysis of a University Library; final report and research project Buckland, M.K. The establishment of nine new universities in the 1960'...",4,system analysi univers librari ; final report research project buckland k establish nine new univers 1960 's provok highli stimul re-examin natur ...
4,5,"A Library Management Game: a report on a research project Brophy, P. Although the use of games in professional education has become widespread onl...",5,librari manag game report research project brophi p although use game profession educ becom widespread last decad method use number field mani hun...
...,...,...,...,...
1455,1456,"World Dynamics Forrester, J.W. Over the last several decades interest in economic development, population growth, and the world environment has ex...",1456,world dynam forrest j w last sever decad interest econom develop popul growth world environ expand rapidli world-wid stress increas mani individu ...
1456,1457,"World Trends in Library Education Bramley, G. One of the most significant aspects of the evolution of librarianship in the twentieth century has b...",1457,world trend librari educ bramley g one signific aspect evolut librarianship twentieth centuri emerg librari school potent factor shape new philoso...
1457,1458,"Legal Restrictions on Exploitation of the Patent Monopoly: An Economic Analysis Baxter, W.A. The patent laws confer on a patentee power to exclude...",1458,legal restrict exploit patent monopoli econom analysi baxter w patent law confer patente power exclud other make use sell invent further constitut...
1458,1459,"Language and Thought Poluskin, V.A. This book considers the basic aspects of this complex problem - the historical and social essence of language ...",1459,languag thought poluskin v book consid basic aspect complex problem - histor social essenc languag thought interact histor evolut essenc linguist ...


In [ ]:
queries_df["query"]=queries_df["raw_query"].apply(preprocess)
queries_df

,qid,raw_query,query
0,1,What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from app...,problem concern make descript titl difficulti involv automat retriev articl approxim titl usual relev content articl titl
1,2,"How can actually pertinent data, as opposed to references or entire articles themselves, be retrieved automatically in response to information req...",actual pertin data oppos refer entir articl retriev automat respons inform request
2,3,What is information science? Give definitions where possible.,inform scienc give definit possibl
3,4,Image recognition and any other methods of automatically transforming printed text into computer-ready form.,imag recognit method automat transform print text computer-readi form
4,5,What special training will ordinary researchers and businessmen need for proper information management and unobstructed use of information retriev...,special train ordinari research businessmen need proper inform manag unobstruct use inform retriev system problem like encount
...,...,...,...
107,108,".T A Program for Machine-Mediated Searching .A Toliver, D. A technique of online instruction and assistance to bibliographic data base searchers c...",program machine-medi search toliv techniqu onlin instruct assist bibliograph data base searcher call individu instruct data access ( iida ) develo...
108,109,".T Author Cocitation: A Literature Measure of Intellectual Structure .A White, H.D. Griffith, B.C. It is shown that the mapping of a particular a...",author cocit literatur measur intellectu structur white h griffith b c shown map particular area scienc case inform scienc done use author unit an...
109,110,".T Progress in Documentation. Word Processing: An Introduction and Appraisal .A Whitehead, J. The ""Office of the Future,"" ""Office Technology,"" ""W...",progress document word process introduct apprais whitehead j `` offic futur `` `` offic technolog `` `` word process `` `` electron mail `` `` ele...
110,111,".T Document Clustering Using an Inverted File Approach .A Willett, P. An automated document clustering procedure is described which does not requi...",document cluster use invert file approach willett p autom document cluster procedur describ requir use inter-docu similar matrix independ order do...


## Indexing

In [ ]:
if not pt.started():
  # In this lab, we need to specify that we start PyTerrier with PRF enabled
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

terrier-assemblies 5.9 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done
terrier-prf -SNAPSHOT jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8



In [ ]:
indexer = pt.DFIndexer("./DatasetIndex", overwrite=True)
# index the text, record the docnos as metadata
index_ref = indexer.index(doc_df["processed_text"], doc_df["docno"])
print(index_ref.toString())

./DatasetIndex/data.properties


In [ ]:
index = pt.IndexFactory.of(index_ref)

In [ ]:
for kv in index.getLexicon():
  print("%s -> %s " % (kv.getKey(), kv.getValue().toString()))

Streaming output truncated to the last 5000 lines.
district -> term2641 Nt=2 TF=2 maxTF=1 @{0 28098 7} 
disturb -> term5266 Nt=1 TF=1 maxTF=1 @{0 28102 7} 
dittmar -> term5101 Nt=1 TF=1 maxTF=1 @{0 28105 3} 
diver -> term1539 Nt=17 TF=20 maxTF=2 @{0 28107 7} 
divert -> term1865 Nt=1 TF=1 maxTF=1 @{0 28135 2} 
divi -> term2164 Nt=18 TF=24 maxTF=3 @{0 28137 0} 
divid -> term1185 Nt=21 TF=29 maxTF=5 @{0 28165 0} 
divis -> term3958 Nt=1 TF=1 maxTF=1 @{0 28195 4} 
divorc -> term3340 Nt=1 TF=1 maxTF=1 @{0 28197 6} 
dk -> term6212 Nt=2 TF=2 maxTF=1 @{0 28200 0} 
dna -> term7157 Nt=1 TF=2 maxTF=2 @{0 28203 0} 
dobrov -> term6370 Nt=2 TF=2 maxTF=1 @{0 28205 7} 
doc -> term5283 Nt=1 TF=1 maxTF=1 @{0 28210 3} 
doctor -> term3610 Nt=3 TF=4 maxTF=2 @{0 28212 7} 
doctrin -> term1023 Nt=2 TF=3 maxTF=2 @{0 28220 0} 
docu -> term1578 Nt=3 TF=4 maxTF=2 @{0 28223 3} 
document -> term60 Nt=250 TF=563 maxTF=10 @{0 28230 2} 
documentalist -> term1994 Nt=8 TF=8 maxTF=1 @{0 28427 3} 
documentari -> term398 Nt

In [ ]:
def find_unique_terms(doc_df):
    docs_dictionary = {}
    i = 0

    for docs in doc_df['processed_text']:
        doc_id = f'doc{i}'
        docs_dictionary[doc_id] = word_tokenize(docs)
        i += 1

    terms_frequency = defaultdict(int)

    for doc_id, doc in docs_dictionary.items():
        for term in doc:
            terms_frequency[term] += 1

    unique_terms = []
    for term, frequency in terms_frequency.items():
        if frequency == 1:
            unique_terms.append(term)

    return unique_terms

unique_terms = find_unique_terms(doc_df)
print("Unique words are:")
for term in unique_terms:
    print(f'Term: {term}')


Unique words are:
Term: comaromi
Term: eighteenth
Term: healthi
Term: 6300
Term: 104
Term: slogan
Term: how-to-do-it
Term: characteristc
Term: abstract-publish
Term: computer-prepar
Term: pleasant
Term: remodel
Term: unsuccess
Term: mistak
Term: farber
Term: apprenticeship
Term: engend
Term: love
Term: admonit
Term: fiction
Term: belles-lett
Term: hyman
Term: data-gath
Term: 1890
Term: self-real
Term: self-indulg
Term: untutor
Term: objection
Term: self-educ
Term: kindr
Term: ligu
Term: bibliothequ
Term: europeenn
Term: recherch
Term: luxembourg
Term: 'third
Term: 'intellectu
Term: sussex
Term: ad695049
Term: wooster
Term: northeastern
Term: waltham
Term: temer
Term: cuddli
Term: ad-669204
Term: cohen
Term: star
Term: matthi
Term: classification-seri
Term: pz3
Term: pz4
Term: viiia
Term: ixa
Term: reclassifi
Term: delv
Term: tortuou
Term: 50-year
Term: enclos
Term: parenthes
Term: voigt
Term: caught
Term: bibliotherapi
Term: 1961b
Term: post-1970
Term: herdan
Term: multifari
Term: idio

In [ ]:
def calculate_term_frequency(doc_df):
    # Create docs_dictionary
    docs_dictionary = {f"doc{i}": word_tokenize(doc) for i, doc in enumerate(doc_df['processed_text'])}

    # Initialize an empty defaultdict to store terms freq
    terms_frequency = defaultdict(int)

    # Iterate over each document in docs_dictionary and update terms_frequency
    for doc in docs_dictionary.values():
        for term in doc:
            terms_frequency[term] += 1

    return terms_frequency

terms_frequency = calculate_term_frequency(doc_df)
for term, frequency in terms_frequency.items():
    print(f'Term: {term}, Freq: {frequency}')


Streaming output truncated to the last 5000 lines.
Term: exalt, Freq: 1
Term: depress, Freq: 3
Term: expos, Freq: 5
Term: censorship, Freq: 6
Term: fisk, Freq: 1
Term: threaten, Freq: 3
Term: uphold, Freq: 2
Term: freedom-to-read, Freq: 1
Term: forgotten, Freq: 1
Term: courag, Freq: 1
Term: moon, Freq: 2
Term: lucidli, Freq: 1
Term: abli, Freq: 2
Term: lester, Freq: 1
Term: 237, Freq: 1
Term: buyer, Freq: 1
Term: mann, Freq: 2
Term: 1967-68, Freq: 1
Term: self-contain, Freq: 6
Term: 239, Freq: 3
Term: junior, Freq: 9
Term: riri, Freq: 1
Term: preinstruct, Freq: 1
Term: authorit, Freq: 1
Term: four-year, Freq: 4
Term: lower-divis, Freq: 1
Term: acceler, Freq: 6
Term: placement, Freq: 1
Term: hermer, Freq: 1
Term: recapitul, Freq: 1
Term: one-and-a-half, Freq: 1
Term: work-rel, Freq: 2
Term: 119, Freq: 7
Term: carter, Freq: 3
Term: government, Freq: 9
Term: awesom, Freq: 1
Term: young, Freq: 10
Term: veteran, Freq: 1
Term: emot, Freq: 2
Term: scrupul, Freq: 1
Term: honesti, Freq: 1
Term:


## Query Processing


In [ ]:
tfidf_retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"},num_results=10)

In [ ]:
def search_and_get_doc_numbers(query, retrieval_model, doc_df):
    preprocessed_query = preprocess(query)
    results = retrieval_model.search(preprocessed_query)
    doc_numbers = results.head(2)['docno'].tolist()
    top_answers = doc_df[doc_df["docno"].isin(doc_numbers)]['processed_text']
    return results, doc_numbers

# Assuming you have a retrieval model named tfidf_retr and a DataFrame named doc_df
query = "imag recognit method automat transform print text computer-readi form"
results, doc_numbers = search_and_get_doc_numbers(query, tfidf_retr, doc_df)
print("Results:")
display(results)
print("Document Numbers of Top 2 Docs:", doc_numbers)


Results:


,qid,docid,docno,rank,score,query
0,1,178,179,0,11.673065,imag recognit method automat transform print text computer-readi form
1,1,76,77,1,11.627949,imag recognit method automat transform print text computer-readi form
2,1,419,420,2,11.552853,imag recognit method automat transform print text computer-readi form
3,1,600,601,3,11.278835,imag recognit method automat transform print text computer-readi form
4,1,174,175,4,11.208871,imag recognit method automat transform print text computer-readi form
5,1,564,565,5,11.183050,imag recognit method automat transform print text computer-readi form
6,1,789,790,6,11.090077,imag recognit method automat transform print text computer-readi form
7,1,745,746,7,10.434095,imag recognit method automat transform print text computer-readi form
8,1,580,581,8,8.959012,imag recognit method automat transform print text computer-readi form
9,1,660,661,9,8.958975,imag recognit method automat transform print text computer-readi form


Document Numbers of Top 2 Docs: ['179', '77']


In [ ]:
def get_top_answers(doc_df, doc_numbers):
    top_answers = doc_df[doc_df["docno"].isin(doc_numbers)]['processed_text']
    return top_answers


results, doc_numbers = search_and_get_doc_numbers(query, tfidf_retr, doc_df)
doc_numbers_plus_one = [str(int(doc_no) + 1) for doc_no in doc_numbers]
top_answer = get_top_answers(doc_df, doc_numbers_plus_one)
print(top_answer)

77     on-lin access inform nsf aid index catalog bennet john l ad document collect index choos represent make evid content document relat document alrea...
179    autom information-retriev system ( ir ) belonogov g g 1973 easiest appli mechan autom transmiss accumul search inform autom also extend cover comp...
Name: processed_text, dtype: object


## Query Expansion

In [ ]:
# retrieval model
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

In [ ]:
results = bm25.search(query)
results

,qid,docid,docno,rank,score,query
0,1,600,601,0,19.794452,imag recognit method automat transform print text computer-readi form
1,1,419,420,1,19.636726,imag recognit method automat transform print text computer-readi form
2,1,76,77,2,19.186244,imag recognit method automat transform print text computer-readi form
3,1,178,179,3,19.070187,imag recognit method automat transform print text computer-readi form
4,1,564,565,4,18.363168,imag recognit method automat transform print text computer-readi form
...,...,...,...,...,...,...
675,1,700,701,675,1.533774,imag recognit method automat transform print text computer-readi form
676,1,496,497,676,1.514508,imag recognit method automat transform print text computer-readi form
677,1,605,606,677,1.508193,imag recognit method automat transform print text computer-readi form
678,1,1214,1215,678,1.489561,imag recognit method automat transform print text computer-readi form


In [ ]:
rm3_expander = pt.rewrite.RM3(index,fb_terms=10, fb_docs=100)
rm3_qe = bm25 >> rm3_expander
expanded_query = rm3_qe.search(query).iloc[0]["query"]

expanded_query

'applypipeline:off languag^0.037381265 procedur^0.015200921 automat^0.172321796 recognit^0.066666670 semant^0.019717088 form^0.066666670 imag^0.066666670 text^0.144425005 chemic^0.022112163 classif^0.033681288 print^0.066666670 readi^0.000000000 transform^0.066666670 comput^0.066666670 abstract^0.030021580 method^0.066666670 machin^0.024030082 word^0.034442134'

In [ ]:
for s in expanded_query.split()[1:]:
  print(s)

print("\n" + query)

languag^0.037381265
procedur^0.015200921
automat^0.172321796
recognit^0.066666670
semant^0.019717088
form^0.066666670
imag^0.066666670
text^0.144425005
chemic^0.022112163
classif^0.033681288
print^0.066666670
readi^0.000000000
transform^0.066666670
comput^0.066666670
abstract^0.030021580
method^0.066666670
machin^0.024030082
word^0.034442134

imag recognit method automat transform print text computer-readi form


In [ ]:
import pandas as pd

def get_expanded_query_results(bm25, expanded_query, doc_df):
    expanded_query_formatted = ' '.join(expanded_query.split()[1:])
    results_wqe = bm25.search(expanded_query_formatted)

    top_results_original = results[['docid','score']][0:5].add_suffix('_1')
    top_results_expanded = results_wqe[['docid','score']][0:5].add_suffix('_2')

    concatenated_results = pd.concat([top_results_original, top_results_expanded], axis=1).fillna('')

    top_documents = doc_df[['Text']][doc_df['docno'].isin(results_wqe['docno'].loc[0:5].tolist())]

    return concatenated_results, top_documents


concatenated_results, top_documents = get_expanded_query_results(bm25, expanded_query, doc_df)
print("    BeforeExpansion       After Expansion")
display(concatenated_results)
print("Top Documents:")
display(top_documents)


    BeforeExpansion       After Expansion


,docid_1,score_1,docid_2,score_2
0,600,19.794452,419,16.300104
1,419,19.636726,178,15.703576
2,76,19.186244,174,15.054546
3,178,19.070187,76,14.568501
4,564,18.363168,789,14.473921


Top Documents:


,Text
76,"The Effectiveness of Automatically Generated Weights and Links in Mechanical Indexing Artandi, Susan Wolf, Edward H. Work concerned with the stati..."
174,"Automatic Information, Organization and Retrieval Salton, G. Information retrieval is a field concerned with the structure, analysis, organization..."
178,"Automatic information, organization and retrieval Salton, G. This book deals with the computer processing of large information files, with special..."
419,"The Automatic Creation of Literature Abstracts Luhn, H.P. Experts of technical papers and magazine articles that serve the purposes of conventiona..."
660,"Is Automatic Classification a Reasonable Application of Statistical Analysis of Text? Doyle, Lauren B. The statistical approach to the analysis of..."
789,"Computer Indexing of Medical Articles - Project Medico Artandi, Susan An automatic indexing method is described in which index tags for documents ..."


CBOW

In [ ]:
corpus = []
for _, row in doc_df.iterrows():
    words = row['Text'].split()  # Split
    corpus.append(words)

print(corpus)

[['18', 'Editions', 'of', 'the', 'Dewey', 'Decimal', 'Classifications', 'Comaromi,', 'J.P.', 'The', 'present', 'study', 'is', 'a', 'history', 'of', 'the', 'DEWEY', 'Decimal', 'Classification.', 'The', 'first', 'edition', 'of', 'the', 'DDC', 'was', 'published', 'in', '1876,', 'the', 'eighteenth', 'edition', 'in', '1971,', 'and', 'future', 'editions', 'will', 'continue', 'to', 'appear', 'as', 'needed.', 'In', 'spite', 'of', 'the', "DDC's", 'long', 'and', 'healthy', 'life,', 'however,', 'its', 'full', 'story', 'has', 'never', 'been', 'told.', 'There', 'have', 'been', 'biographies', 'of', 'Dewey', 'that', 'briefly', 'describe', 'his', 'system,', 'but', 'this', 'is', 'the', 'first', 'attempt', 'to', 'provide', 'a', 'detailed', 'history', 'of', 'the', 'work', 'that', 'more', 'than', 'any', 'other', 'has', 'spurred', 'the', 'growth', 'of', 'librarianship', 'in', 'this', 'country', 'and', 'abroad.', '1'], ['Use', 'Made', 'of', 'Technical', 'Libraries', 'Slater,', 'M.', 'This', 'report', 'is', 

In [ ]:
# Train CBOW model
model = Word2Vec(sentences=corpus,
                 sg=0,                 #indicates the Cbow model
                 vector_size=100,
                 window=2,
                 min_count=1,
                 workers=4,
                 epochs=20)


In [ ]:
# Get word embeddings
word_embeddings = model.wv

In [ ]:
print(word_embeddings['history'])

[ 4.5797188e-02  3.8723725e-01 -1.0333857e-01  8.9634918e-02
  2.2656208e-01 -1.8677422e-01  3.9987001e-01  7.3968208e-01
 -8.1548923e-01 -2.0683740e-01 -1.4119425e-01 -7.1239954e-01
 -7.4594982e-02  3.1233379e-01  8.9981392e-02 -2.2937335e-01
  5.2540660e-01 -9.3043566e-02 -1.7696021e-01 -1.0855137e+00
  4.6448463e-01  1.4367962e-01  9.5851892e-01 -2.6995870e-01
 -4.3447819e-01  8.7070897e-02 -3.1110945e-01 -2.2065924e-01
 -3.6332372e-01  1.3180947e-01  2.8939101e-01 -2.0231436e-01
  5.6072950e-01 -9.3547750e-01 -3.5126099e-01  8.7913567e-01
  2.0475313e-01  6.9016732e-02 -3.1597418e-01 -4.1803011e-01
  9.0676725e-02 -2.8391403e-01  5.3472171e-04  2.5788715e-01
  3.6006466e-01 -7.3425747e-02 -5.2228707e-01 -3.2340273e-01
  3.2571697e-01  9.3342967e-02  9.4690517e-02 -3.2749948e-01
 -1.0198519e-02  3.5713203e-03 -6.4440742e-02  1.4838213e-01
  3.2066804e-01 -7.8998581e-02 -2.6513937e-01 -1.4410424e-01
  2.8160292e-01 -2.4966389e-01 -2.3055214e-01 -6.5121397e-02
 -2.7708682e-01  5.87499

## User Interface and Evaluation

In [ ]:
from google.colab.output import eval_js
print (eval_js("google.colab.kernel.proxyPort(5000)"))

https://680t235650w-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:
from flask import Flask, render_template, request
import pandas as pd
import pyterrier as pt

app = Flask(__name__)

# Define your search function
def search(query, bm25_model):
    results = bm25_model.search(query)
    return results

# Define a function to evaluate search engine performance
def evaluate_search_engine(query, search_func, doc_df, bm25):
    start_time = time.time()
    results = search_func(query, bm25)
    end_time = time.time()
    response_time = end_time - start_time

    # Calculate retrieval accuracy (assuming relevant documents contain query term)
    relevant_docs = doc_df[doc_df['Text'].str.contains(query, case=False)]
    if len(results) > 0:
        precision = len(set(results['docno']) & set(relevant_docs['docno'])) / len(results)
        recall = len(set(results['docno']) & set(relevant_docs['docno'])) / len(relevant_docs)
        accuracy = precision * recall  # F1 score
    else:
        precision = 0.00
        recall = 0.00
        accuracy = 0.00

    return results, accuracy, response_time

# Combined User Interface and Evaluation Function
def search_and_evaluate(query):
    bm25 = pt.BatchRetrieve(index, wmodel="BM25")

    # Call search and evaluate function
    results, accuracy, response_time = evaluate_search_engine(query, search, doc_df, bm25)

    # Extract relevant documents for display
    top_documents = doc_df[doc_df['docno'].isin(results['docno'])]

    # Prepare documents for display
    combined_docs_str = ""
    for idx, row in top_documents.iterrows():
        combined_docs_str += f"Document ID: {row['docno']} - Text: {row['Text']}<br><br>"

    return combined_docs_str, accuracy, response_time

@app.route('/', methods=['GET', 'POST'])
def calculate():
    result = ""
    accuracy = 0.00
    response_time = 0.00

    if request.method == 'POST':
        query = request.form.get('query')
        if query:
            result, accuracy, response_time = search_and_evaluate(query)

    return render_template('index.html', result=result, accuracy=accuracy, response_time=response_time)


if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [11/May/2024 23:29:36] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/May/2024 23:29:36] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/May/2024 23:29:39] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/May/2024 23:29:40] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/May/2024 23:29:56] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/May/2024 23:29:56] "GET /favicon.ico HTTP/1.1" 404 -


##TRY

In [ ]:
# import pandas as pd

# def query_expansion(query):
#     # Assuming bm25 and doc_df are defined outside this function
#     expanded_query_formatted = ' '.join(query.split()[1:])
#     results_wqe = bm25.search(expanded_query_formatted)

#     print("   Before Expansion    After Expansion")
#     print(pd.concat([results[['docid', 'score']][0:5].add_suffix('_1'),
#                      results_wqe[['docid', 'score']][0:5].add_suffix('_2')], axis=1).fillna(''))

#     # Let's check the tweets text for the top 5 retrieved tweets
#     top_results = doc_df[['Text']][doc_df['docno'].isin(results_wqe['docno'].loc[0:5].tolist())]
#     return top_results


In [ ]:


# # Define a function to perform search using BM25 and return documents according to scores
# def search(query, bm25_model):
#     results = bm25_model.search(query)
#     return results

# # Define a function to evaluate search engine performance
# def evaluate_search_engine(query, search_func, doc_df):
#     start_time = time.time()
#     results = search_func(query, bm25)
#     end_time = time.time()
#     response_time = end_time - start_time

#     # Display search results and evaluate performance
#     print("\nSearch results:")
#     if len(results) == 0:
#         print("No results found.")
#         precision = 0.00
#         recall = 0.00
#     else:
#         for idx, row in results.iterrows():
#             print(f"Document ID: {row['docno']} ")
#         # Calculate retrieval accuracy (assuming relevant documents contain query term)
#         relevant_docs = doc_df[doc_df['Text'].str.contains(query, case=False)]
#         if len(relevant_docs) > 0:
#             precision = len(set(results['docno']) & set(relevant_docs['docno'])) / len(results)
#             recall = len(set(results['docno']) & set(relevant_docs['docno'])) / len(relevant_docs)
#         else:
#             precision = 0.00
#             recall = 0.00

#     print(f"Precision: {precision:.2f}, Recall: {recall:.2f}")
#     print(f"Response Time: {response_time:.4f} seconds\n")

# #import pandas as pd

# import pandas as pd

# def query_expansion(query):
#     # Assuming bm25 and doc_df are defined outside this function
#     expanded_query_formatted = ' '.join(query.split()[1:])
#     results_wqe = bm25.search(expanded_query_formatted)

#     top_results_original = results[['docid', 'score']][:].add_suffix('_1')
#     top_results_expanded = results_wqe[['docid', 'score']][:].add_suffix('_2')

#     concatenated_results = pd.concat([top_results_original, top_results_expanded], axis=1).fillna('')

#     # Extract the expanded document IDs
#     expanded_doc_ids = results_wqe['docid'].loc[:].tolist()

#     return concatenated_results, expanded_doc_ids


In [ ]:
# def get_expanded_query_result(query):
#     results_wqe = bm25.search(query)

#     top_results_original = results[['docid', 'score']].add_suffix('_1')
#     top_results_expanded = results_wqe[['docid', 'score']].add_suffix('_2')

#     top_documents = doc_df[['docno', 'Text']][doc_df['docno'].isin(results_wqe['docno'].tolist())]

#     combined_docs_str = ""

#     for idx, row in top_documents.iterrows():
#         combined_docs_str += f"Document ID: {row['docno']} - Text: {row['Text']}\n \n"

#     return combined_docs_str


In [ ]:
# # Define a function to evaluate search engine performance
# def evaluate_search_engine(query, search_func, doc_df):
#     start_time = time.time()
#     results = search_func(query, bm25)
#     end_time = time.time()
#     response_time = end_time - start_time

#     # Calculate retrieval accuracy (assuming relevant documents contain query term)
#     relevant_docs = doc_df[doc_df['Text'].str.contains(query, case=False)]
#     if len(results) > 0:
#         precision = len(set(results['docno']) & set(relevant_docs['docno'])) / len(results)
#         recall = len(set(results['docno']) & set(relevant_docs['docno'])) / len(relevant_docs)
#         accuracy = precision * recall  # F1 score
#     else:
#         precision = 0.00
#         recall = 0.00
#         accuracy = 0.00

#     return accuracy, response_time

In [ ]:
# def search_and_evaluate(query):
#     # Perform search operation using BM25
#     results_wqe = bm25.search(query)

#     # Extract relevant documents based on search results
#     top_documents = doc_df[['docno', 'Text']][doc_df['docno'].isin(results_wqe['docno'].tolist())]

#     # Combine document IDs and text into a string
#     combined_docs_str = ""
#     for idx, row in top_documents.iterrows():
#         combined_docs_str += f"Document ID: {row['docno']} - Text: {row['Text']}\n\n"

#     # Calculate retrieval accuracy and response time
#     start_time = time.time()
#     results = search(query, bm25)
#     end_time = time.time()
#     response_time = end_time - start_time

#     relevant_docs = doc_df[doc_df['Text'].str.contains(query, case=False)]
#     if len(results) > 0:
#         precision = len(set(results['docno']) & set(relevant_docs['docno'])) / len(results)
#         recall = len(set(results['docno']) & set(relevant_docs['docno'])) / len(relevant_docs)
#         accuracy = precision * recall  # F1 score
#     else:
#         precision = 0.00
#         recall = 0.00
#         accuracy = 0.00

#     return combined_docs_str, accuracy, response_time


## UI

In [ ]:

# # Define your search function
# def search(query, doc_df):
#     # Perform search operation based on query and return relevant results
#     results = doc_df[doc_df['Text'].str.contains(query, case=False)]
#     return results

# # Basic command-line interface
# def search_interface(doc_df):
#     print("Welcome to the Search Engine!")
#     while True:
#         query = input("Enter your search query (type 'exit' to quit): ")
#         if query.lower() == 'exit':
#             print("Exiting the Search Engine.")
#             break

#         # Call the search function with the user query
#         results = search(query, doc_df)

#         # Display search results
#         if len(results) == 0:
#             print("No results found.")
#         else:
#             print("\nSearch results:")
#             for idx, row in results.iterrows():
#                 print(f"Document ID: {row['docno']} - {row['Text']}")
#             print()

# # Example usage
# search_interface(doc_df)


## Evaluation

In [ ]:
# import pandas as pd
# import time


# # Define your search function
# def search(query, doc_df):
#     # Perform search operation based on query and return relevant results
#     results = doc_df[doc_df['Text'].str.contains(query, case=False)]
#     return results

# # Define a function to evaluate search engine performance
# def evaluate_search_engine(search_func, doc_df):
#     while True:
#         query = input("Enter your search query (type 'exit' to quit): ")
#         if query.lower() == 'exit':
#             print("Exiting the search engine evaluation.")
#             break

#         start_time = time.time()
#         results = search_func(query, doc_df)
#         end_time = time.time()
#         response_time = end_time - start_time

#         print("Search results:")
#         if len(results) == 0:
#             print("No results found.")
#         else:
#             for idx, row in results.iterrows():
#                 print(f"Document ID: {row['docno']} - {row['Text']}")
#         print(f"Response Time: {response_time:.4f} seconds\n")

# # Example usage
# evaluate_search_engine(search, doc_df)


## Combination

In [ ]:
# import pandas as pd
# import time
# import pyterrier as pt





# # Define a function to perform search using BM25 and return documents according to scores
# def search(query, bm25_model):
#     results = bm25_model.search(query)
#     return results

# # Define a function to evaluate search engine performance
# def evaluate_search_engine(query, search_func, doc_df):
#     start_time = time.time()
#     results = search_func(query, bm25)
#     end_time = time.time()
#     response_time = end_time - start_time

#     # Display search results and evaluate performance
#     print("\nSearch results:")
#     if len(results) == 0:
#         print("No results found.")
#         precision = 0.00
#         recall = 0.00
#     else:
#         for idx, row in results.iterrows():
#             print(f"Document ID: {row['docno']} ")
#         # Calculate retrieval accuracy (assuming relevant documents contain query term)
#         relevant_docs = doc_df[doc_df['Text'].str.contains(query, case=False)]
#         if len(relevant_docs) > 0:
#             precision = len(set(results['docno']) & set(relevant_docs['docno'])) / len(results)
#             recall = len(set(results['docno']) & set(relevant_docs['docno'])) / len(relevant_docs)
#         else:
#             precision = 0.00
#             recall = 0.00

#     print(f"Precision: {precision:.2f}, Recall: {recall:.2f}")
#     print(f"Response Time: {response_time:.4f} seconds\n")

# # User Interface and Evaluation Function
# def search_and_evaluate(doc_df):
#     print("Welcome to the Search Engine!")
#     while True:
#         query = input("Enter your search query (type 'exit' to quit): ")
#         if query.lower() == 'exit':
#             print("Exiting the Search Engine.")
#             break

#         # Call the search function with the user query
#         evaluate_search_engine(query, search, doc_df)

# # Example usage
# search_and_evaluate(doc_df)
